# mongodb query building: NOTES

MongoDB queries are extremely verbose in Python.

They are presented either as nested dictionaries or,
even more deeply nested SON objects (to preserve order).
Resulting queries are hard to write and hard to read.

## status

After surveying the available builders and testing one,
I was unsatisfied, so I drafter my own, MQuery (below).

However, while working with it a came to the conclusion
that string parsing might be a better approach. This
led me to find PQL (link below) -- if I continued with
this project, I would probably start by testing PQL.

## survey

### query builders

For writing queries, on possibility is a builder. There are
three options:

- https://github.com/AureaRobotics (fork with Python2 -> Python3 fix)
  clear, but doesn't implement comparison operators AND OR etc at all. 
- https://github.com/ziffusion/python-mongo-query 
  much better -- a bit strange, but INIT isn't required, and handles AND OR etc. correctly.
  written for Python 2.
- MQuery, written here, below

### string parsers

A string parser changes a DSL into a pymongo query object.
(e.g. as strings written in the style of native python statements)

-  PQL Python Query Language https://github.com/alonho/pql  
   translates python expressions to MongoDB queries
    
```
>>> import pql
>>> pql.find("a > 1 and b == 'foo' or not c.d == False")
{'$or': [{'$and': [{'a': {'$gt': 1}}, {'b': 'foo'}]}, {'$not': {'c.d': False}}]}
```


## MQuery: a pymongo query builder

In [20]:
import json
import os
import pprint
from pymongo import MongoClient 
from pymongo.errors import DuplicateKeyError, InvalidDocument

import sys
sys.path.insert(0, '/home/jovyan/utils/preprocessing/')
from libs.fuzzyhasher.fuzzyhasher import FuzzyHasher
from libs.zipeditor.zipeditor import ZipEditor, zip_scanner, zip_scanner_excludedirs, ZipProcessor
from we1s_utils.ziputils import BatchJSONUploader

class MQuery:
    """"""
    def __init__(self, query=''):
        if not query:
            self.query = {}
        self.pp = pprint.PrettyPrinter(indent=4, compact=False)
        
    def _clause(self, key, val, compare='regex', logic='and', query=''):
        logic = '$' + logic
        if not query:
            query = self.query
        if type(val) in [list, tuple, set]:
            query_list = []
            for val_entry in val:
                query_list.append({ key : { '$' + compare :'.*' + val_entry + '.*' } })
            query.setdefault(logic, []).extend(query_list)
        else:
            query[key] = { '$' + compare :'.*' + val + '.*' }
        return self
    
    def AND(self, key, val):
        return self._clause(key, val, logic='and')

    def NOR(self, key, val):
        return self._clause(key, val, logic='nor')

#     def NOT(self, key, val):
#         return self._clause(key, val, logic='not')

    def print(self):
        self.pp.pprint(self.query)
        return self

    def OR(self, key, val):
        return self._clause(key, val, logic='or')

    def TOP(self, key, val):
        if type(val) in [list, tuple, set]:
            if len(val)==1:
                val=val[0]
                return self._clause(key, val, logic='and')
            else:
                raise ValueError("TOP takes one key:val")
        return self._clause(key, val)

# {pub:  'Mail|Times',
#  name: 'liberal'|'humanities'
# (pub:(Mail|Times))
# name:


In [21]:
# connect with MongoDB URI:
client = MongoClient('mongodb://mongo/')
db = client['we1s']
corpus = db['Corpus']


## MQuery unit tests

In [38]:
import sys
import unittest

class MQueryInputs(unittest.TestCase):

    def test_TOP(self):
        """Create and print a simple query builder"""
        MQuery().TOP(key='name', val='humanities').print()

    def test_staged(self):
        # create a more complex querybuilder
        qb = MQuery()
        # call methods to add query criteria,
        # then use the result query dict with mongodb:
        # TOP takes a single required term
        qb.TOP('name', 'body')        
        # pretty-print the result
        qb.print()

    def test_TOP_implicit_and(self):
        # Additional TOP keys may be added
        # with additional calls -- they are implicitly AND requirements
        # and all must be fulfilled by search results
        qb = MQuery()
        qb.TOP('pub', 'Times').print()
        # These operations can be chained
        qb = MQuery().TOP('name', 'plural').TOP('pub', 'Mail').print()
        # However, changing the same TOP key replaces it:
        qb.TOP('pub', 'Mail').print()
        # ...so don't use qb.TOP to add multiple criteria for the same key
        qb = MQuery().TOP('pub', 'Times').TOP('pub', 'Mail').print()
        # oops! {'pub': {'$regex': '.*Mail.*'}}
        qb = MQuery().TOP('pub', 'Times').TOP('pub', 'Mail').print()
        # oops! {'pub': {'$regex': '.*Mail.*'}}

    
suite = unittest.TestLoader().loadTestsFromTestCase(MQueryInputs)
unittest.TextTestRunner(verbosity=4,stream=sys.stderr).run(suite)

test_TOP (__main__.MQueryInputs)
Create and print a simple query builder ... ok
test_TOP_implicit_and (__main__.MQueryInputs) ... ok
test_staged (__main__.MQueryInputs) ... 

{'name': {'$regex': '.*humanities.*'}}
{'pub': {'$regex': '.*Times.*'}}
{'name': {'$regex': '.*plural.*'}, 'pub': {'$regex': '.*Mail.*'}}
{'name': {'$regex': '.*plural.*'}, 'pub': {'$regex': '.*Mail.*'}}
{'pub': {'$regex': '.*Mail.*'}}
{'pub': {'$regex': '.*Mail.*'}}
{'name': {'$regex': '.*body.*'}}


ok

----------------------------------------------------------------------
Ran 3 tests in 0.015s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [26]:
# create a more complex querybuilder

qb = MQuery()

# call methods to add query criteria,
# then use the result query dict with mongodb

# TOP takes a single required term
qb.TOP('name', 'body')

# pretty-print the result
qb.print()

# use the query
one_doc = corpus.find_one(qb.query, {'content':0})
print('qb result:\n', one_doc, '\n')

{'name': {'$regex': '.*body.*'}}
qb result:
 None 



In [27]:
# Additional TOP keys may be added
# with additional calls -- they are implicitly AND requirements
# and all must be fulfilled by search results

qb.TOP('pub', 'Times').print()

# These operations can be chained

qb = MQuery().TOP('name', 'plural').TOP('pub', 'Mail').print()

# However, changing the same TOP key replaces it:

qb.TOP('pub', 'Mail').print()

# ...so don't use qb.TOP to add multiple criteria for the same key

qb = MQuery().TOP('pub', 'Times').TOP('pub', 'Mail').print()
# oops! {'pub': {'$regex': '.*Mail.*'}}

qb = MQuery().TOP('pub', 'Times').TOP('pub', 'Mail').print()
# oops! {'pub': {'$regex': '.*Mail.*'}}


{'name': {'$regex': '.*body.*'}, 'pub': {'$regex': '.*Times.*'}}
{'name': {'$regex': '.*plural.*'}, 'pub': {'$regex': '.*Mail.*'}}
{'name': {'$regex': '.*plural.*'}, 'pub': {'$regex': '.*Mail.*'}}
{'pub': {'$regex': '.*Mail.*'}}
{'pub': {'$regex': '.*Mail.*'}}


In [28]:
# use AND for multiple criteria on one key

# it can take a list of terms

qb = MQuery()
qb.AND('name', ['2016', 'humanities'])
qb.print()

# AND can be built with multiple calls

qb = MQuery()
qb.AND('name', '2016')
qb.AND('name', 'humanities')
qb.print()

# AND can also be chained

qb = MQuery().AND('name', '2016').AND('name', 'humanities').print()

# qb.OR( 'pub',  ['Mail', 'Times'])
# qb.TOP('name', ['body']) # if not in a list, becomes a top-level term.

# qb.AND('name', val=['humanities','liberal'])
# qb.print()

# note that the heirarchy of terms is stable,
# but the printed order of individual terms may not relate to code order
# -- prettyprint alphabetizes, but dicts are unordered.

{   '$and': [   {'name': {'$regex': '.*2016.*'}},
                {'name': {'$regex': '.*humanities.*'}}]}
{'name': {'$regex': '.*humanities.*'}}
{'name': {'$regex': '.*humanities.*'}}


In [29]:
# use queries to compare document counts
count1  = corpus.count_documents(MQuery().AND('pub', val='Mail').query)
count2 = corpus.count_documents(MQuery().AND('pub', val=['Times']).query)
print(count1)
print(count2)

0
0


In [30]:
# pass an entry or a single entry list
# -- different query form, same results

query = MQuery().TOP('pub', val='Mail').query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().TOP('pub', val=['Mail']).query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().AND('pub', val='Mail').query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().AND('pub', val=['Mail']).query
count = corpus.count_documents(query)
print(count, query)

0 {'pub': {'$regex': '.*Mail.*'}}
0 {'pub': {'$regex': '.*Mail.*'}}
0 {'pub': {'$regex': '.*Mail.*'}}
0 {'$and': [{'pub': {'$regex': '.*Mail.*'}}]}


In [31]:
query = MQuery().OR('pub', val='Mail').query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().OR('pub', val='Times').query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().OR('pub', val=['Mail']).query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().OR('pub', val=['Times']).query
count = corpus.count_documents(query)
print(count, query)

0 {'pub': {'$regex': '.*Mail.*'}}
0 {'pub': {'$regex': '.*Times.*'}}
0 {'$or': [{'pub': {'$regex': '.*Mail.*'}}]}
0 {'$or': [{'pub': {'$regex': '.*Times.*'}}]}


In [32]:
# AND and OR

# Look at individual counts

query = MQuery().TOP('name', val='liberal').query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().TOP('name', val='humanities').query
count = corpus.count_documents(query)
print(count, query)

# AND: document must have both

query = MQuery().AND('name', val=['liberal', 'humanities']).query
count = corpus.count_documents(query)
print(count, query)

# OR: document may have either

query = MQuery().OR('name', val=['liberal', 'humanities']).query
count = corpus.count_documents(query)
print(count, query)

# AND: document must have both

query = MQuery().AND('pub', val=['Mail', 'Times']).query
count = corpus.count_documents(query)
print(count, query)

# OR: document may have either

query = MQuery().OR('pub', val=['Mail', 'Times']).query
count = corpus.count_documents(query)
print(count, query)

# AND is restrictive, OR is inclusive.

query = MQuery().AND('pub', val=['Mail', 'Times']).OR('name', val=['liberal', 'humanities']).query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().OR('pub', val=['Mail', 'Times']).AND('name', val=['liberal', 'humanities']).query
count = corpus.count_documents(query)
print(count, query)
query = MQuery().OR('pub', val=['Mail', 'Times']).OR('name', val=['liberal', 'humanities']).query
count = corpus.count_documents(query)
print(count, query)

# 

print("\ncount:", count)
one_doc = corpus.find_one(query, {'content':0})
print("one article:\n", one_doc)

0 {'name': {'$regex': '.*liberal.*'}}
0 {'name': {'$regex': '.*humanities.*'}}
0 {'$and': [{'name': {'$regex': '.*liberal.*'}}, {'name': {'$regex': '.*humanities.*'}}]}
0 {'$or': [{'name': {'$regex': '.*liberal.*'}}, {'name': {'$regex': '.*humanities.*'}}]}
0 {'$and': [{'pub': {'$regex': '.*Mail.*'}}, {'pub': {'$regex': '.*Times.*'}}]}
0 {'$or': [{'pub': {'$regex': '.*Mail.*'}}, {'pub': {'$regex': '.*Times.*'}}]}
0 {'$and': [{'pub': {'$regex': '.*Mail.*'}}, {'pub': {'$regex': '.*Times.*'}}], '$or': [{'name': {'$regex': '.*liberal.*'}}, {'name': {'$regex': '.*humanities.*'}}]}
0 {'$or': [{'pub': {'$regex': '.*Mail.*'}}, {'pub': {'$regex': '.*Times.*'}}], '$and': [{'name': {'$regex': '.*liberal.*'}}, {'name': {'$regex': '.*humanities.*'}}]}
0 {'$or': [{'pub': {'$regex': '.*Mail.*'}}, {'pub': {'$regex': '.*Times.*'}}, {'name': {'$regex': '.*liberal.*'}}, {'name': {'$regex': '.*humanities.*'}}]}

count: 0
one article:
 None


## MQuery API dev notes


### query operators

https://docs.mongodb.com/manual/reference/operator/query/

The biggest thing for making MQuery more flexible would be passing through operators correctly.

#### TO ADD

```
Comparison
$eq	Matches values that are equal to a specified value.
$gt	Matches values that are greater than a specified value.
$gte	Matches values that are greater than or equal to a specified value.
$in	Matches any of the values specified in an array.
$lt	Matches values that are less than a specified value.
$lte	Matches values that are less than or equal to a specified value.
$ne	Matches all values that are not equal to a specified value.
$nin	Matches none of the values specified in an array.

Logical
$and	Joins query clauses with a logical AND returns all documents that match the conditions of both clauses.
$not	Inverts the effect of a query expression and returns documents that do not match the query expression.
$nor	Joins query clauses with a logical NOR returns all documents that fail to match both clauses.
$or	Joins query clauses with a logical OR returns all documents that match the conditions of either clause.

Element
$exists	Matches documents that have the specified field.

Evaluation
$regex	Selects documents where values match a specified regular expression.
$text	Performs text search.

Array
$all	Matches arrays that contain all elements specified in the query.
$elemMatch	Selects documents if element in the array field matches all the specified $elemMatch conditions.
```

#### LOW PRIORITY API
    
```
Element
$type	Selects documents if a field is of the specified type.

Evaluation
$expr	Allows use of aggregation expressions within the query language.
$jsonSchema	Validate documents against the given JSON Schema.
$mod	Performs a modulo operation on the value of a field and selects documents with a specified result.
$where	Matches documents that satisfy a JavaScript expression.

Geospatial
Name	Description
$geoIntersects	Selects geometries that intersect with a GeoJSON geometry. The 2dsphere index supports $geoIntersects.
$geoWithin	Selects geometries within a bounding GeoJSON geometry. The 2dsphere and 2d indexes support $geoWithin.
$near	Returns geospatial objects in proximity to a point. Requires a geospatial index. The 2dsphere and 2d indexes support $near.
$nearSphere	Returns geospatial objects in proximity to a point on a sphere. Requires a geospatial index. The 2dsphere and 2d indexes support $nearSphere.

Array
$size	Selects documents if the array field is a specified size.

Bitwise
$bitsAllClear	Matches numeric or binary values in which a set of bit positions all have a value of 0.
$bitsAllSet	Matches numeric or binary values in which a set of bit positions all have a value of 1.
$bitsAnyClear	Matches numeric or binary values in which any bit from a set of bit positions has a value of 0.
$bitsAnySet	Matches numeric or binary values in which any bit from a set of bit positions has a value of 1.

Comments
Name	Description
$comment	Adds a comment to a query predicate.
Projection Operators
Name	Description
$	Projects the first element in an array that matches the query condition.
$elemMatch	Projects the first element in an array that matches the specified $elemMatch condition.
$meta	Projects the document’s score assigned during $text operation.
$slice	Limits the number of elements projected from an array. Supports skip and limit slices.
```

format ideas

```
qb.OR(('pub', 'val'), ('pub', 'val'))

{'or' : [('pub', 'val'), ('name', 'val')]}
{'and' : [('pub', 'val'), ('name', 'val')]}
{'or' : [('pub', '$eq', 'val'), ('name', 'val')]}
{'and' : [('pub', 'val'), ('name', 'val')]}

a = QB('key', 'val')
b = QB('key2', '$eq', 'val')
c = QB.AND(a, b)
d = QB.OR(QB('key', 'val'), QB('key2', 'val'))
e = QB.OR(('key', 'val'), ('key2', 'val'))

mb.or()

mb.q('pub', ['val1', 'val2'])
mb.q('pub', '$eq', ['val1', 'val2'])
```


